In [1]:
import pandapower as pp
import pandapower.networks as nw
import pandas as pd
import itertools

In [2]:
# Pandapower N-2 anaylsis code


def run_power_flow(net):
    pp.runpp(net)
    return net.res_bus, net.res_line


def get_line_buses(net, line_index):
    # Fetches the from_bus and to_bus of a line
    from_bus = net.line.at[line_index, "from_bus"] + 1
    to_bus = net.line.at[line_index, "to_bus"] + 1
    return (from_bus, to_bus)


def n_2_analysis(net):
    original_line_status = net.line.in_service.copy()
    contingency_results = []

    # Generate all combinations of two lines
    line_combinations = itertools.combinations(net.line.index, 2)

    for line1, line2 in line_combinations:
        # Fetch bus information for the lines
        line1_buses = get_line_buses(net, line1)
        line2_buses = get_line_buses(net, line2)

        # Disconnect lines
        net.line.in_service.at[line1] = False
        net.line.in_service.at[line2] = False

        # Run power flow
        try:
            pp.runpp(net)
            # Check if power flow converged
            converged = net["converged"]
            bus_voltages = net.res_bus.vm_pu.tolist()
            line_loadings = net.res_line.loading_percent.tolist()
            contingency_results.append(
                (line1_buses, line2_buses, converged, bus_voltages, line_loadings)
            )
        except pp.LoadflowNotConverged:
            # Handle cases where power flow does not converge
            contingency_results.append((line1_buses, line2_buses, False, [], []))

        # Restore line status
        net.line.in_service.at[line1] = True
        net.line.in_service.at[line2] = True

    return pd.DataFrame(
        contingency_results,
        columns=["Line 1", "Line 2", "Converged", "Bus Voltages", "Line Loadings"],
    )


# Create and setup network

net = nw.case39()

# Run initial power flow
initial_bus_res, initial_line_res = run_power_flow(net)

initial_line_res

# Perform N-2 analysis
contingency_results = n_2_analysis(net)
print("\nN-2 Contingency Analysis Results:")
contingency_results


N-2 Contingency Analysis Results:


,Line 1,Line 2,Converged,Bus Voltages,Line Loadings
0,"(1, 2)","(1, 39)",True,"[nan, 1.0470984755759538, 1.0287450950238293, ...","[nan, nan, 90.65245335150077, 43.2997775595129..."
1,"(1, 2)","(2, 3)",True,"[1.0276593166370516, 1.063612196751277, 0.9867...","[0.0, 10.425824995353286, 0.0, 49.409243519047..."
2,"(1, 2)","(2, 25)",True,"[1.0276593166370516, 1.0523494171161103, 1.029...","[0.0, 10.425824995353262, 55.79062226242827, 0..."
3,"(1, 2)","(3, 4)",True,"[1.0276593166370516, 1.0546371104579668, 1.044...","[0.0, 10.425824995353288, 82.15654967831938, 3..."
4,"(1, 2)","(3, 18)",True,"[1.0276593166370516, 1.0447831407025585, 1.023...","[0.0, 10.42582499535318, 93.46824137060874, 44..."
...,...,...,...,...,...
590,"(26, 27)","(26, 29)",True,"[1.034746065124118, 1.0380121733866667, 1.0172...","[32.93129853887004, 11.239197099499897, 105.56..."
591,"(26, 27)","(28, 29)",True,"[1.034631935516403, 1.0376068135229863, 1.0169...","[32.65442166776521, 11.056791517513858, 104.35..."
592,"(26, 28)","(26, 29)",True,"[1.039656224918178, 1.0468126364176105, 1.0253...","[20.380604720203582, 7.928506598985528, 52.207..."
593,"(26, 28)","(28, 29)",True,"[1.0359849289231413, 1.0414526469349674, 1.023...","[33.39171519537976, 11.84444136962072, 74.1425..."
